## Поиск ключевых слов по топикам и разбиение по ним постов с реддита

In [1]:
import praw
import json
from tqdm import tqdm
import nltk
import collections

In [2]:
reddit = praw.Reddit(client_id='i9OxQyEl0ikmDQ', client_secret='hg0viho18o2lUIbMT3bSpYhN1KGXTA', user_agent='Scraping')

In [3]:
topics = ["Animals", "news", "food", "travel", "sports", "science", "Music", "Games", "books", "movies", "CelebrityGossip"]

In [4]:
subreddit2texts = {}

for topic in tqdm(topics):
    texts = []
    hot_posts = reddit.subreddit(topic).hot(limit=5000)
    for post in hot_posts:
        texts.append(post.title + " " + post.selftext)
    subreddit2texts[topic] = texts

100%|██████████| 11/11 [01:27<00:00,  7.98s/it]


In [5]:
topics2skills = {"Animals": "dff_animals_skill",
                 "news": "news_api_skill",
                 "food": "dff_food_skill",
                 "travel": "dff_travel_skill",
                 "sports": "dff_sport_skill",
                 "science": "dff_science_skill",
                 "Music": "dff_music_skill",
                 "Games": "game_cooperative_skill",
                 "books": "book_skill",
                 "movies": "dff_movie_skill",
                 "CelebrityGossip": 'dff_gossip_skill'}

#'entertain': 'dff_gaming_skill'

In [6]:
key100words = {}
key1000words = {}

for key, value in topics2skills.items():
    c = collections.Counter()
    sentences = " ".join(subreddit2texts[key])
    tokens = nltk.word_tokenize(sentences)
    for word in tokens:
        if len(word) > 2:
            c[word.lower()] += 1
    key100words[value] = [word[0] for word in c.most_common(100)]
    #key1000words[value] = [word[0] for word in c.most_common(1000)]

In [7]:
unique_words = {}

for skill, word_set in key100words.items():
    other_word_set = []
    for key, value in key100words.items():
        if key != skill:
            other_word_set += value
    unique_words[skill] = list(set(word_set).difference(set(other_word_set)))

In [8]:
for k, v in unique_words.items():
    print(k, len(v))

dff_animals_skill 38
news_api_skill 57
dff_food_skill 98
dff_travel_skill 19
dff_sport_skill 50
dff_science_skill 49
dff_music_skill 32
game_cooperative_skill 27
book_skill 9
dff_movie_skill 12
dff_gossip_skill 35


In [10]:
unique_words["dff_animals_skill"]

['cute',
 'wolves',
 'ghost',
 'gray',
 'dog',
 'united',
 'wildlife',
 'facts',
 'maybe',
 'little',
 'welfare',
 'providing',
 'identify',
 'kind',
 'information',
 'park',
 'got',
 'backyard',
 'pets',
 'bird',
 'giant',
 'plant',
 'mantids',
 'saw',
 'animal',
 'cat',
 'last',
 'pics',
 'things',
 'morning',
 'every',
 'rare',
 'pet',
 'species',
 'animals',
 'today',
 'wild',
 'goats']

In [11]:
with open("users_reddit_665.json", "r") as content:
    loaded_data = json.load(content)

In [12]:
parsed_submissions = loaded_data["parsed_submissions"]
user2subreddits = loaded_data["user2subreddits"]
subreddit2submission = loaded_data["subreddit2submission"]

In [48]:
user2subreddits

{'Relaxed-Sheep': {'submissions': [{'np17x3': 'Pets'},
   {'np1b52': 'DOG'},
   {'npfr50': 'Animals'},
   {'npdnub': 'pcmasterrace'},
   {'np7yfa': 'Pets'},
   {'nna17o': 'weed'},
   {'nky0nv': 'Christianity'},
   {'nkmj0q': 'Monitors'},
   {'nkotkt': 'nvidia'},
   {'nkmfi5': 'pcmasterrace'}],
  'comments': [{'h03r1a1': 'Pets'},
   {'h04lfbl': 'Pets'},
   {'h03pn6c': 'Pets'},
   {'h03j8k3': 'Pets'},
   {'h04gauy': 'Pets'},
   {'h04bldb': 'Pets'},
   {'h04gwxo': 'pcmasterrace'},
   {'h04gik9': 'pcmasterrace'},
   {'h04qx4j': 'Pets'},
   {'h04qrl3': 'Pets'}]},
 'beltran_randy': {'submissions': [{'np3qg3': 'news'},
   {'nogm3w': 'politics'},
   {'nogys8': 'news'},
   {'nobpiq': 'news'},
   {'nonuqk': 'news'},
   {'nongue': 'news'},
   {'nofquf': 'worldnews'},
   {'nofp23': 'news'},
   {'nofg71': 'news'},
   {'nl4z7b': 'news'}],
  'comments': [{'gzxvi0z': 'politics'},
   {'gzy12pk': 'politics'},
   {'gzzd6qb': 'news'},
   {'gzxjwdr': 'politics'},
   {'gzxgmw1': 'politics'},
   {'gzxejl7': 

In [13]:
scenario_skills = list(topics2skills.values())

search_topics = {"animal": "dff_animals_skill",
                 "news": "news_api_skill",
                 "food": "dff_food_skill", 
                 "travel": "dff_travel_skill",
                 "sport": "dff_sport_skill",
                 "science": "dff_science_skill",
                 "music": "dff_music_skill",
                 "game": "game_cooperative_skill",
                 "book": "book_skill",
                 "movie": "dff_movie_skill",
                 "gossip": "dff_gossip_skill"}

In [66]:
scenario_skills

['dff_animals_skill',
 'news_api_skill',
 'dff_food_skill',
 'dff_travel_skill',
 'dff_sport_skill',
 'dff_science_skill',
 'dff_music_skill',
 'game_cooperative_skill',
 'book_skill',
 'dff_movie_skill',
 'dff_gossip_skill']

In [55]:
subreddit2topic = {}
subreddit2candidate = {}

for subred, texts in subreddit2submission.items():
        
    current_count = {skill: 0 for skill in scenario_skills}
    
    all_texts = " ".join([parsed_submissions[text]['submission']['title'].lower() + " " + 
                          parsed_submissions[text]['submission']['text'].lower() for text in texts]).lower()
        
    for topic in search_topics.keys():
        
        if topic in subred.lower():
            subreddit2topic[subred] = search_topics[topic]
            break
            
                    
        else:
            
            for key_word in unique_words[search_topics[topic]]:
                if key_word == subred.lower():
                    subreddit2topic[subred] = search_topics[topic]
                    break
                    
                elif key_word in subred.lower():
                    if len(subreddit2candidate.get(subred, "")) == 0:
                        subreddit2candidate[subred] = [(key_word, search_topics[topic])]
                    else:
                        subreddit2candidate[subred].append((key_word, search_topics[topic]))
                
                elif key_word in all_texts:
                    current_count[search_topics[topic]] += all_texts.count(key_word)
                    
    for s, kw in subreddit2candidate.items():
        if subreddit2topic.get(s, "") == "":
            cur_max = ("", "")
            print("S", s)
            for w in kw:
                print(w)
                if len(w[0]) > len(cur_max[0]):
                    cur_max = w
            print("CUR", cur_max)
            subreddit2topic[s] = cur_max[1]
                    
    if subred == "ramen":
        print(subreddit2topic[subred])
                    
    current_count = {k: v/len(unique_words[k]) for k, v in current_count.items()}
            
    if subreddit2topic.get(subred, "") == "":
        if subred == "ramen":
            print("ramen")
        sorted_current_count = sorted(current_count.items(), key=lambda x: x[1])
        #print(subred)
        #print(sorted_current_count)
        subreddit2topic[subred] = sorted_current_count[-1][0]

S pcmasterrace
('race', 'dff_sport_skill')
CUR ('race', 'dff_sport_skill')
S TuckedInPuppies
('pie', 'dff_food_skill')
CUR ('pie', 'dff_food_skill')
S 4Runner
('run', 'dff_sport_skill')
CUR ('run', 'dff_sport_skill')
dff_food_skill
S kindle
('kind', 'dff_animals_skill')
CUR ('kind', 'dff_animals_skill')
S AdvertiseEverything
('every', 'dff_animals_skill')
('ever', 'dff_sport_skill')
CUR ('every', 'dff_animals_skill')
S SubscribeToMe
('tom', 'dff_gossip_skill')
CUR ('tom', 'dff_gossip_skill')
S todayilearned
('today', 'dff_animals_skill')
CUR ('today', 'dff_animals_skill')
S LateStageCapitalism
('ate', 'dff_food_skill')
('test', 'dff_sport_skill')
('age', 'dff_science_skill')
CUR ('test', 'dff_sport_skill')
S juststart
('start', 'dff_sport_skill')
CUR ('start', 'dff_sport_skill')
S OnePiece
('pie', 'dff_food_skill')
CUR ('pie', 'dff_food_skill')
S femalefashionadvice
('advice', 'dff_travel_skill')
CUR ('advice', 'dff_travel_skill')
S CrimeScene
('scene', 'dff_movie_skill')
CUR ('scene',

S ebonyhomemade
('homemade', 'dff_food_skill')
('home', 'dff_sport_skill')
('made', 'dff_movie_skill')
CUR ('homemade', 'dff_food_skill')
S Eredivisie
('red', 'dff_food_skill')
CUR ('red', 'dff_food_skill')
S AskMen
('men', 'dff_science_skill')
CUR ('men', 'dff_science_skill')
S nevertellmetheodds
('ever', 'dff_sport_skill')
CUR ('ever', 'dff_sport_skill')
S teenagers
('teen', 'news_api_skill')
('age', 'dff_science_skill')
CUR ('teen', 'news_api_skill')
S unitedkingdom
('united', 'dff_animals_skill')
CUR ('united', 'dff_animals_skill')
S WatchHolUpDie
('watch', 'dff_movie_skill')
CUR ('watch', 'dff_movie_skill')
S RoastMe
('roast', 'dff_food_skill')
CUR ('roast', 'dff_food_skill')
S twilightstruggle
('light', 'dff_science_skill')
CUR ('light', 'dff_science_skill')
S entertainment
('men', 'dff_science_skill')
CUR ('men', 'dff_science_skill')
S orangecounty
('orange', 'dff_food_skill')
CUR ('orange', 'dff_food_skill')
S MealPics
('pics', 'dff_animals_skill')
CUR ('pics', 'dff_animals_ski

S theydidthemath
('hey', 'dff_music_skill')
CUR ('hey', 'dff_music_skill')
S vagabond
('bond', 'dff_movie_skill')
CUR ('bond', 'dff_movie_skill')
S flashlight
('light', 'dff_science_skill')
CUR ('light', 'dff_science_skill')
S urbanclimbing
('ban', 'news_api_skill')
CUR ('ban', 'news_api_skill')
S deadmalls
('dead', 'news_api_skill')
CUR ('dead', 'news_api_skill')
S DeathStranding
('death', 'news_api_skill')
CUR ('death', 'news_api_skill')
S TrueAskReddit
('red', 'dff_food_skill')
CUR ('red', 'dff_food_skill')
S changemyview
('change', 'dff_science_skill')
CUR ('change', 'dff_science_skill')
S latterdaysaints
('days', 'dff_travel_skill')
CUR ('days', 'dff_travel_skill')
S PandR
('pan', 'dff_food_skill')
CUR ('pan', 'dff_food_skill')
S JulieAndThePhantoms
('tom', 'dff_gossip_skill')
CUR ('tom', 'dff_gossip_skill')
S BisexualTeens
('teen', 'news_api_skill')
CUR ('teen', 'news_api_skill')
S NewGirl
('girl', 'news_api_skill')
CUR ('girl', 'news_api_skill')
S AskWomen
('men', 'dff_science_s

S camphalfblood
('camp', 'dff_gossip_skill')
CUR ('camp', 'dff_gossip_skill')
S FORTNITESKINBATTLES
('nba', 'dff_sport_skill')
CUR ('nba', 'dff_sport_skill')
S homecooking
('home', 'dff_sport_skill')
CUR ('home', 'dff_sport_skill')
S REDDITTBAY
('red', 'dff_food_skill')
CUR ('red', 'dff_food_skill')
S RedditbayPro
('red', 'dff_food_skill')
CUR ('red', 'dff_food_skill')
S RedditBay_Official
('red', 'dff_food_skill')
('official', 'game_cooperative_skill')
CUR ('official', 'game_cooperative_skill')
S realredditbay
('red', 'dff_food_skill')
CUR ('red', 'dff_food_skill')
S AnythingGoesUltimate
('ate', 'dff_food_skill')
CUR ('ate', 'dff_food_skill')
S FreeKarma4You
('free', 'game_cooperative_skill')
CUR ('free', 'game_cooperative_skill')
S PopularOnYouTube
('pop', 'dff_music_skill')
CUR ('pop', 'dff_music_skill')
S metalgearsolid
('metal', 'dff_music_skill')
('gear', 'game_cooperative_skill')
CUR ('metal', 'dff_music_skill')
S MetalMemes
('metal', 'dff_music_skill')
CUR ('metal', 'dff_music_

S biggerthanherhead
('head', 'news_api_skill')
CUR ('head', 'news_api_skill')
S LebaneseGirls
('girl', 'news_api_skill')
('ban', 'news_api_skill')
CUR ('girl', 'news_api_skill')
S exxxchange
('change', 'dff_science_skill')
CUR ('change', 'dff_science_skill')
S cuteasfuckbutclothed
('cute', 'dff_animals_skill')
('lot', 'book_skill')
CUR ('cute', 'dff_animals_skill')
S cleavage
('age', 'dff_science_skill')
CUR ('age', 'dff_science_skill')
S IRLgirls
('girl', 'news_api_skill')
CUR ('girl', 'news_api_skill')
S AskRedditAfterDark
('red', 'dff_food_skill')
CUR ('red', 'dff_food_skill')
S PetiteGoneWild
('pet', 'dff_animals_skill')
('wild', 'dff_animals_skill')
CUR ('wild', 'dff_animals_skill')
S phgonewild
('wild', 'dff_animals_skill')
CUR ('wild', 'dff_animals_skill')
S IndianCelebScenes
('scene', 'dff_movie_skill')
CUR ('scene', 'dff_movie_skill')
S BikiniBodies
('dies', 'news_api_skill')
CUR ('dies', 'news_api_skill')
S WatchItForThePlot
('lot', 'book_skill')
('watch', 'dff_movie_skill')


S sololeveling
('level', 'game_cooperative_skill')
CUR ('level', 'game_cooperative_skill')
S MachineLearning
('mac', 'dff_food_skill')
CUR ('mac', 'dff_food_skill')
S algotrading
('got', 'dff_animals_skill')
CUR ('got', 'dff_animals_skill')
S investmentdata
('men', 'dff_science_skill')
CUR ('men', 'dff_science_skill')
S flowers
('lower', 'dff_science_skill')
('low', 'dff_science_skill')
CUR ('lower', 'dff_science_skill')
S FolkPunk
('punk', 'dff_music_skill')
('folk', 'dff_music_skill')
CUR ('punk', 'dff_music_skill')
S HumansBeingBros
('man', 'news_api_skill')
('humans', 'dff_science_skill')
('human', 'dff_science_skill')
CUR ('humans', 'dff_science_skill')
S Windows10
('win', 'dff_sport_skill')
CUR ('win', 'dff_sport_skill')
S FinancialPlanning
('plan', 'dff_travel_skill')
('planning', 'dff_travel_skill')
CUR ('planning', 'dff_travel_skill')
S Affiliatemarketing
('ate', 'dff_food_skill')
CUR ('ate', 'dff_food_skill')
S PokemonFriendExchange
('change', 'dff_science_skill')
CUR ('chang

S carparkreviews
('park', 'dff_animals_skill')
('review', 'game_cooperative_skill')
CUR ('review', 'game_cooperative_skill')
S PraiseTheCameraMan
('man', 'news_api_skill')
CUR ('man', 'news_api_skill')
S Dollhouses
('house', 'dff_music_skill')
CUR ('house', 'dff_music_skill')
S skywind
('win', 'dff_sport_skill')
CUR ('win', 'dff_sport_skill')
S indonesia
('don', 'dff_music_skill')
CUR ('don', 'dff_music_skill')
S ihaveihaveihavereddit
('red', 'dff_food_skill')
CUR ('red', 'dff_food_skill')
S radiohead
('head', 'news_api_skill')
CUR ('head', 'news_api_skill')
S TeenWolf
('teen', 'news_api_skill')
CUR ('teen', 'news_api_skill')
S HistoryWhatIf
('history', 'dff_sport_skill')
CUR ('history', 'dff_sport_skill')
S MonsterHunterMeta
('monster', 'game_cooperative_skill')
CUR ('monster', 'game_cooperative_skill')
S orangetheory
('orange', 'dff_food_skill')
CUR ('orange', 'dff_food_skill')
S AskGayMen
('men', 'dff_science_skill')
CUR ('men', 'dff_science_skill')
S RidiculousRealEstate
('ate', 'd

In [59]:
subreddit2topic

{'Pets': 'dff_animals_skill',
 'DOG': 'dff_animals_skill',
 'Animals': 'dff_animals_skill',
 'pcmasterrace': 'dff_sport_skill',
 'weed': 'dff_travel_skill',
 'Christianity': 'book_skill',
 'Monitors': 'book_skill',
 'nvidia': 'book_skill',
 'news': 'news_api_skill',
 'politics': 'news_api_skill',
 'worldnews': 'news_api_skill',
 'food': 'dff_food_skill',
 'TuckedInPuppies': 'dff_food_skill',
 '4Runner': 'dff_sport_skill',
 'Toyota': 'game_cooperative_skill',
 'ramen': 'dff_food_skill',
 'pitbulls': 'dff_animals_skill',
 'kindle': 'dff_animals_skill',
 'Tacoma': 'dff_travel_skill',
 'AmItheAsshole': 'dff_travel_skill',
 'DutchOvenCooking': 'dff_travel_skill',
 'Cartalk': 'dff_travel_skill',
 'cookingforbeginners': 'book_skill',
 'TravelVlogger': 'dff_travel_skill',
 'Youtube_Support': 'dff_music_skill',
 'Vlogs': 'dff_animals_skill',
 'TravelSolo': 'dff_travel_skill',
 'channelsofyoutube': 'dff_movie_skill',
 'AdvertiseYourVideos': 'dff_animals_skill',
 'advertiseyoutube': 'book_skill',

In [47]:
subreddit2topic["ramen"]

'dff_science_skill'

In [35]:
len(subreddit2topic)

2878

In [31]:
for i in range(5):
    if i == 3:
        print(i)
        break
    print(i)

0
1
2
3


In [60]:
subreddit2topic["InstantRamen"]

'dff_food_skill'

In [77]:
subreddit2topic

{'Pets': 'dff_animals_skill',
 'DOG': 'dff_animals_skill',
 'Animals': 'dff_animals_skill',
 'pcmasterrace': 'dff_sport_skill',
 'weed': 'dff_travel_skill',
 'Christianity': 'book_skill',
 'Monitors': 'book_skill',
 'nvidia': 'book_skill',
 'news': 'news_api_skill',
 'politics': 'dff_science_skill',
 'worldnews': 'news_api_skill',
 'food': 'dff_food_skill',
 'TuckedInPuppies': 'dff_food_skill',
 '4Runner': 'dff_sport_skill',
 'Toyota': 'dff_music_skill',
 'ramen': 'dff_science_skill',
 'pitbulls': 'dff_animals_skill',
 'kindle': 'dff_animals_skill',
 'Tacoma': 'dff_music_skill',
 'AmItheAsshole': 'book_skill',
 'DutchOvenCooking': 'dff_travel_skill',
 'Cartalk': 'dff_science_skill',
 'cookingforbeginners': 'book_skill',
 'TravelVlogger': 'dff_travel_skill',
 'Youtube_Support': 'book_skill',
 'Vlogs': 'dff_animals_skill',
 'TravelSolo': 'dff_travel_skill',
 'channelsofyoutube': 'dff_travel_skill',
 'AdvertiseYourVideos': 'game_cooperative_skill',
 'advertiseyoutube': 'book_skill',
 'Adv

In [61]:
users_emb = []

for user, items in user2subreddits.items():
    emb_names = []
    for el in items['submissions']:
        users_subreddit = list(el.values())[0]
        subreddit_topic = subreddit2topic[users_subreddit]
        emb_names.append(subreddit_topic)
        
    for el in items['comments']:
        users_subreddit = list(el.values())[0]
        subreddit_topic = subreddit2topic[users_subreddit]
        emb_names.append(subreddit_topic)
        
    current_dict = {skill: 0 for skill in scenario_skills}
    current_emb = []
    for el in emb_names:
        if len(el) > 0:
            current_dict[el] += 1
    current_emb = [current_dict[skill]/len(scenario_skills) for skill in scenario_skills]
    users_emb.append(current_emb)

In [62]:
users_emb

[[1.0909090909090908,
  0.0,
  0.0,
  0.09090909090909091,
  0.36363636363636365,
  0.0,
  0.0,
  0.0,
  0.2727272727272727,
  0.0,
  0.0],
 [0.0, 1.4545454545454546, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.18181818181818182,
  0.0,
  0.8181818181818182,
  0.45454545454545453,
  0.18181818181818182,
  0.0,
  0.0,
  0.09090909090909091,
  0.09090909090909091,
  0.0,
  0.0],
 [0.2727272727272727,
  0.0,
  0.0,
  0.45454545454545453,
  0.0,
  0.18181818181818182,
  0.2727272727272727,
  0.0,
  0.18181818181818182,
  0.18181818181818182,
  0.09090909090909091],
 [0.0,
  0.45454545454545453,
  0.0,
  0.9090909090909091,
  0.2727272727272727,
  0.0,
  0.09090909090909091,
  0.0,
  0.09090909090909091,
  0.0,
  0.0],
 [0.18181818181818182,
  0.09090909090909091,
  0.0,
  0.0,
  0.18181818181818182,
  1.0909090909090908,
  0.09090909090909091,
  0.0,
  0.18181818181818182,
  0.0,
  0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
 [0.0,
  0.09090909090909091,
  0.09090

In [64]:
import pickle

In [65]:
with open('reddit_embeddings.pickle', 'wb') as f:
    pickle.dump(users_emb, f)